<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part6/P6_Ch02_CH02_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Moving Average를 활용한 비디오 classification 후처리


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from glob import glob
import os
import tensorflow_hub as hub


In [3]:
LABEL_INT_DICT = np.unique(pd.read_csv('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/ucf11_train_vid.csv')['label'])
LABEL_STR_DICT = { k:v for v, k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_INT_DICT)
pprint(LABEL_STR_DICT)


array(['biking', 'diving', 'golf', 'juggle', 'jumping', 'riding',
       'shooting', 'spiking', 'swing', 'tennis', 'walk'], dtype=object)
{'biking': 0,
 'diving': 1,
 'golf': 2,
 'juggle': 3,
 'jumping': 4,
 'riding': 5,
 'shooting': 6,
 'spiking': 7,
 'swing': 8,
 'tennis': 9,
 'walk': 10}


In [4]:
import os
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model(
    '/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/ucf11_weights/10-0.78.hdf5',
    custom_objects={'KerasLayer': hub.KerasLayer}
)

model.build([None, 256,256,3])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              5919312   
                                                                 
 dense (Dense)               (None, 11)                14091     
                                                                 
Total params: 5933403 (22.63 MB)
Trainable params: 5872795 (22.40 MB)
Non-trainable params: 60608 (236.75 KB)
_________________________________________________________________


In [ ]:
csv_path = '/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/ucf11_valid_vid.csv'
valid_df = pd.read_csv(csv_path)

idx = 21
elem = valid_df.iloc[idx]

cap = cv2.VideoCapture(
    elem['file_path']
)
fourcc = cv2.VideoWriter_fourcc(*'MJPG') # 비디오 인코더
writer = cv2.VideoWriter('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/ori.avi', fourcc, 30, (256,256), True)

while True :
    ret, frame =cap.read()
    if not ret :
        break
    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_ = frame_.astype('float32')
    frame_ = frame_ /255.

    pred = model.predict(frame_[np.newaxis, ...])
    label = LABEL_INT_DICT[np.argmax(pred)]

    frame = cv2.resize(frame, (256, 256))
    cv2.putText(frame, label, (50,45), cv2.FONT_HERSHEY_SIMPLEX, #frame에 label 덮기 위치, 폰트, 폰트사이즈, 색, 굵기
                1.5, (0, 255, 0), 5)
    writer.write(frame)

cap.release()
writer.release()

In [ ]:
# Moving average 를 사용해 fully connect를 막는다.
cap = cv2.VideoCapture(
    elem['file_path']
)
fourcc = cv2.VideoWriter_fourcc(*'MJPG') # 비디오 인코더
writer = cv2.VideoWriter('/content/drive/MyDrive/딥러닝 정주행/P6_Ch02.동영상처리실습/roll.avi', fourcc, 30, (256,256), True)

queue =[]
while True :
    ret, frame =cap.read()
    if not ret :
        break
    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_ = frame_.astype('float32')
    frame_ = frame_ /255.

    pred = model.predict(frame_[np.newaxis, ...])
    queue.append(pred)

    results = np.array(queue).mean(axis =0)
    label = LABEL_INT_DICT[np.argmax(results)]

    frame = cv2.resize(frame, (256, 256))
    cv2.putText(frame, label, (50,45), cv2.FONT_HERSHEY_SIMPLEX, #frame에 label 덮기 위치, 폰트, 폰트사이즈, 색, 굵기
                1.5, (0, 255, 0), 5)
    writer.write(frame)

cap.release()
writer.release()